In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cleverhans
%matplotlib inline
from cleverhans.model import Model
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils_mnist import data_mnist
import math

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import sgld_tf

Instructions for updating:
Use the retry module or similar alternatives.


/home/henripal/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
X_train, Y_train, X_test, Y_test = data_mnist(train_start=0,
                                                  train_end=60000,
                                                  test_start=0,
                                                  test_end=10000)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
X_train shape: (60000, 28, 28, 1)
X_test shape: (10000, 28, 28, 1)


In [6]:
to_run = {'optimizers': ['sgd', 'sgd', 'sgld', 'psgld', 'ksgld'],
         'names': ['sgd', 'sgda', 'sgld', 'psgld', 'ksgld'],
         'bayes_n': [1, 50, 50, 50, 50],
         'lr': [0.01, 0.01, 0.02, 0.02, 0.02, 0.02]}
batch_size = 512
n_epochs = 20
directory = '/tmp/tf/sgd_mine/'

In [7]:
for i, name in enumerate(to_run['names']):
    tf.reset_default_graph()
    tf.logging.set_verbosity(tf.logging.ERROR)
    x, y, preds, logits, loss, train_step, saver, sess, adv_preds = sgld_tf.make_graph(to_run['optimizers'][i],
                                                                                      to_run['lr'][i])
    sgld_tf.train(n_epochs, sess, batch_size, X_train, Y_train, X_test, Y_test, x,
                 y, preds, train_step, loss, directory)
    model_files = sgld_tf.ordered_chkpoints_from_dir(directory)
    model_files = model_files[-to_run['bayes_n'][i]:]
    ev, p, l = sgld_tf.evaluate(sess, batch_size, X_test, Y_test, x, y, preds)
    print('spot model: {}\t test acc: {}'.format(name, ev))
    ev = sgld_tf.bayes_evaluate(model_files, directory, sess, batch_size, X_test, Y_test,
                                x, y, preds)
    print('bayes model: {}\t test acc: {}'.format(name, ev))
    ev = sgld_tf.bayes_evaluate(model_files, directory, sess, batch_size, X_test, Y_test,
                                x, y, adv_preds)
    print('bayes model: {}\t adv test acc: {}'.format(name, ev))

epoch:  0
eval acc 0.6595
epoch:  1
eval acc 0.82
epoch:  2
eval acc 0.8836
epoch:  3
eval acc 0.9054
epoch:  4
eval acc 0.9175
epoch:  5
eval acc 0.9266
epoch:  6
eval acc 0.9324
epoch:  7
eval acc 0.9366
epoch:  8
eval acc 0.9411
epoch:  9
eval acc 0.9449
epoch:  10
eval acc 0.9476
epoch:  11
eval acc 0.9497
epoch:  12
eval acc 0.9516
epoch:  13
eval acc 0.9533
epoch:  14
eval acc 0.9544
epoch:  15
eval acc 0.9553
epoch:  16
eval acc 0.9572
epoch:  17
eval acc 0.9581
epoch:  18
eval acc 0.9597
epoch:  19
eval acc 0.9607
spot model: sgd	 test acc: 0.9607
bayes model: sgd	 test acc: 0.9607
bayes model: sgd	 adv test acc: 0.0287
epoch:  0
eval acc 0.693
epoch:  1
eval acc 0.8279
epoch:  2
eval acc 0.8876
epoch:  3
eval acc 0.9107
epoch:  4
eval acc 0.9233
epoch:  5
eval acc 0.931
epoch:  6
eval acc 0.938
epoch:  7
eval acc 0.9424
epoch:  8
eval acc 0.946
epoch:  9
eval acc 0.9494
epoch:  10
eval acc 0.9524
epoch:  11
eval acc 0.9557
epoch:  12
eval acc 0.9577
epoch:  13
eval acc 0.9593
